In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
from scipy.ndimage import convolve
from skimage import feature, io
from tqdm import tqdm

In [ ]:
# directory containing CGL simulations near the transition to turbulence
SRC_DIR = '/path/to/datasets/vary_near_critical' # CHANGE THIS!!!

In [ ]:
def slice_polar_defects(phi):
    '''Find polar-order defects in phase-field timeslice

    Parameters
    ----------
    phi : np.ndarray
        timeslice of a 2D phase field
        encoded on a uint8 scale [-pi, pi) -> [0, 256)
    
    Returns
    -------
    dd : float
        density of defects in phi
    '''
    # convert to complex representation
    phases = np.array(phi)/255.0*2.0*np.pi-np.pi
    cis = np.exp(1j*phases)

    # sum 2-by-2 neighborhoods to path-integrate angles
    kernel = np.ones((2, 2))
    cis_sum_mag = np.abs(convolve(cis, kernel, mode='wrap'))
    
    # polar-order defects are sites where surrounding cis sum to 0
    defects = feature.peak_local_max(4.0-cis_sum_mag, threshold_abs=1.0)

    return len(defects)/np.prod(phi.shape) # number of defects

In [ ]:
def series_polar_defects(fn):
    '''Find time-averaged polar-order defect density

    Parameters
    ----------
    fn : str
        path to phase-field video
    
    Returns
    -------
    mu_dd : float
        time-averaged defect number
    '''
    phis = io.imread(fn)
    timesteps = phis.shape[0]

    integrated_density = 0
    for phi in phis:
        integrated_density += slice_polar_defects(phi)
    
    return integrated_density/timesteps

In [ ]:
# find time-averaged defect densities of each simulation
density_dict = {}
for bn in tqdm(os.listdir(SRC_DIR)):
    fn = os.path.join(SRC_DIR, bn)
    mu_dd = series_polar_defects(fn)
    density_dict[bn] = mu_dd

In [ ]:
# find mean defect density at each value of c2
c2_density_mu_SE = {}
for c2 in np.linspace(1.13, 1.21, num=9):
    c2_densities = []
    c2_str = ('c2_%.2f'%c2).replace('.', '')
    for key in density_dict.keys():
        if c2_str in key: c2_densities += [density_dict[key]]
    c2_density_mu_SE[c2_str] = (np.mean(c2_densities),
                                np.std(c2_densities, ddof=1)/np.sqrt(len(c2_densities)))
c2_density_mu_SE